In [1]:
import zarr
import xarray as xr
import time
import dask

In [2]:
queue='qt72h'
nb_worker_per_job=40
memory='184GB'
nb_job=2

In [3]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client
import dask.dataframe as dd

cluster = PBSCluster(processes=nb_worker_per_job,cores=nb_worker_per_job, memory=memory,queue=queue,env_extra=['--exclusive'],project='PerfTestPangeo', walltime='00:30:00')
cluster.scale(nb_job*nb_worker_per_job)

c = Client(cluster)

c


/home/ad/alberta/.conda/envs/perf-pangeo/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45919 instead
  http_address["port"], self.http_server.port
Task exception was never retrieved
future: <Task finished coro=<_wrap_awaitable() done, defined at /home/ad/alberta/.conda/envs/perf-pangeo/lib/python3.6/asyncio/tasks.py:530> exception=RuntimeError('Command exited with non-zero exit code.\nExit code: 174\nCommand:\nqsub /tmp/pbs.2586133.admin01/tmp9uvc8_fe.sh\nstdout:\n\nstderr:\nqsub: Access to queue is denied\n\n',)>
Traceback (most recent call last):
  File "/home/ad/alberta/.conda/envs/perf-pangeo/lib/python3.6/asyncio/tasks.py", line 537, in _wrap_awaitable
    return (yield from awaitable.__await__())
  File "/home/ad/alberta/.conda/envs/perf-pangeo/lib/python3.6/site-packages/distributed/deploy/spec.py", line 50, in _
    await self.start()
  File "/home/ad/alb

Client Scheduler: tcp://10.120.41.133:35586 Dashboard: http://10.120.41.133:45919/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [ ]:
from dask.utils import ensure_dict, format_bytes
    
wk = c.scheduler_info()["workers"]

text="Workers= " + str(len(wk))
memory = [w["memory_limit"] for w in wk.values()]
cores = sum(w["nthreads"] for w in wk.values())
text += ", Cores=" + str(cores)
if all(memory):
    text += ", Memory=" + format_bytes(sum(memory))
print(text)


In [ ]:
%time ds=xr.open_zarr('/work/ALT/odatis/eNATL60/zarr/eNATL60-BLBT02-SSH-1h')

In [ ]:
%time mean=ds.sossheig.mean(dim='time_counter')

In [ ]:
%time mean.load()

In [ ]:
c.close()
cluster.close()